# Preprocess Crime data

This notebook will run the `Crime_Download.py` script in the scripts folder to download the crime data to local landing folder

This notebook will then continue preprocessing the data, grouping/counting the crime instance record by location/postcode/time(year)

### Import packages

In [1]:
import pandas as pd


In [2]:
import geopandas as gpd
from fuzzywuzzy import fuzz

import geopandas as gpd
from shapely.geometry import Point

/home/jcc/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Dowload Crime data to landing folder

In [3]:
%run ../scripts/Crime_Download.py

File '../data/landing/crime_stat/crime_stat_2023.xlsx' downloaded successfully.


## Read in our desire table/sheet of crime data 
as table 3 store content of Criminal incidents by principal offence, local government area and postcode or suburb/town - April 2013 to March 2023

In [4]:
# Read the "Table 03" sheet into a DataFrame
sheet_name = "Table 03"
crime_df = pd.read_excel("../data/landing/crime_stat/crime_stat_2023.xlsx", sheet_name=sheet_name)

In [5]:
crime_df

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2023,March,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,1
1,2023,March,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,1
2,2023,March,Alpine,3691,Dederang,B Property and deception offences,B40 Theft,B42 Steal from a motor vehicle,2
3,2023,March,Alpine,3691,Dederang,B Property and deception offences,B40 Theft,B49 Other theft,1
4,2023,March,Alpine,3691,Dederang,D Public order and security offences,D10 Weapons and explosives offences,D11 Firearms offences,1
...,...,...,...,...,...,...,...,...,...
330929,2014,March,Yarriambiack,3489,Tempy,B Property and deception offences,B20 Property damage,B21 Criminal damage,1
330930,2014,March,Yarriambiack,3489,Tempy,B Property and deception offences,B30 Burglary/Break and enter,B322 Non-residential non-aggravated burglary,1
330931,2014,March,Yarriambiack,3489,Tempy,E Justice procedures offences,E20 Breaches of orders,E22 Breach intervention order,1
330932,2014,March,Yarriambiack,3491,Patchewollock,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,1


## Feature selection

In [6]:
# feature selection
crime_df = crime_df[['Year', 'Postcode', 'Suburb/Town Name', 'Incidents Recorded']]
crime_df

,Year,Postcode,Suburb/Town Name,Incidents Recorded
0,2023,3691,Dederang,1
1,2023,3691,Dederang,1
2,2023,3691,Dederang,2
3,2023,3691,Dederang,1
4,2023,3691,Dederang,1
...,...,...,...,...
330929,2014,3489,Tempy,1
330930,2014,3489,Tempy,1
330931,2014,3489,Tempy,1
330932,2014,3491,Patchewollock,1


# Apply agrregation

#### Group by 'Suburb/Town Name', 'Postcode', 'Year' and count the total crime cases by suburbs

In [7]:
# Group by 'Suburb/Town Name', 'Postcode', 'Year'
suburb_crime_counts = crime_df.groupby(['Suburb/Town Name', 'Postcode', 'Year'])['Incidents Recorded'].sum().reset_index()

In [8]:
suburb_crime_counts

,Suburb/Town Name,Postcode,Year,Incidents Recorded
0,Abbeyard,3737,2015,4
1,Abbeyard,3737,2017,1
2,Abbeyard,3737,2019,3
3,Abbeyard,3737,2021,1
4,Abbotsford,3067,2014,793
...,...,...,...,...
23290,Zeerust,3634,2022,2
23291,Zeerust,3634,2023,8
23292,Zumsteins,3401,2018,2
23293,Zumsteins,3401,2019,2


In [9]:
# Save to Raw
suburb_crime_counts.to_csv('../data/raw/suburb_crime_counts.csv', index=False)  # Set index=False to exclude the index column

However, it is important to note that the suburb/Town Name may not be consistent with our SAL name as data is from different provider.
We will find a point geometry for each suburb for later visualisation and merging

# Create a point geometry to the suburb_crime_counts

To do this we will consider combining the crime count data with victoria' postcode data frame.

As postcode data contain postcode (a common feature) and different locality (suburb) which we will be use to match with the "Suburb/Town Name" attribute in our crime data frame by similarity score. 

Most importantly postcode data contains point geometry for the locality.

## Combine crime count with victoria postcode data 
Join by postcode, and select for most similar name for the locality name and Suburb/Town Name from two the data 

In [10]:
# import the postcode data
Postcode_df = pd.read_csv("../data/raw/location_postcode.csv")

In [11]:
Postcode_df

,postcode,locality,SA2_MAINCODE_2016,SA2_NAME_2016,Lat_precise,Long_precise
0,3000,MELBOURNE,206041122.0,Melbourne,-37.815207,144.963937
1,3001,MELBOURNE,206041122.0,Melbourne,-37.813628,144.963058
2,3002,EAST MELBOURNE,206041119.0,East Melbourne,-37.816144,144.980459
3,3003,WEST MELBOURNE,206041127.0,West Melbourne,-37.811450,144.925397
4,3004,MELBOURNE,206041126.0,Southbank,-37.830158,144.980459
...,...,...,...,...,...,...
3532,8120,MELBOURNE,206041122.0,Melbourne,-37.813628,144.963058
3533,8205,MELBOURNE,206041122.0,Melbourne,-37.813628,144.963058
3534,8785,DANDENONG,212041311.0,Dandenong,-37.984781,145.213991
3535,8785,DANDENONG SOUTH,212041311.0,Dandenong,-37.984781,145.213991


Notice how the locality value are all in capital case, we will convert all Suburb/Town Name in the suburb_crime_counts and locality in postcode to lowercase for laters string comparision 

We will not using SA2_NAME and CODE in postcode df as they were from 2016, we will mainly focusing on the locality (suburb) and the lat and long for laters merging 

In [12]:
# Lower case the Suburb/Town Name in crime count data frame
suburb_crime_counts['Suburb/Town Name'] = suburb_crime_counts['Suburb/Town Name'].str.lower()
# Lower case the locality in postcode data
Postcode_df['locality'] = Postcode_df['locality'].str.lower()
# Drop SA2_MAINCODE_2016 and SA2_NAME_2016 in postcode data
Postcode_df = Postcode_df.drop(columns=['SA2_MAINCODE_2016', 'SA2_NAME_2016'])

#### First join suburb_crime_counts and  Postcode_df by left join on postcode number

In [13]:
# Merge the crime data with postcode by postcode number first
crime_count_n_locality= suburb_crime_counts.merge(Postcode_df, left_on='Postcode', right_on='postcode', how='left')

In [14]:
crime_count_n_locality

,Suburb/Town Name,Postcode,Year,Incidents Recorded,postcode,locality,Lat_precise,Long_precise
0,abbeyard,3737,2015,4,3737,abbeyard,-37.021325,146.764065
1,abbeyard,3737,2015,4,3737,barwidgee,-36.509837,146.777018
2,abbeyard,3737,2015,4,3737,buffalo river,-36.618382,146.681521
3,abbeyard,3737,2015,4,3737,dandongadale,-36.911853,146.699920
4,abbeyard,3737,2015,4,3737,gapsted,-36.514069,146.656295
...,...,...,...,...,...,...,...,...
286037,zumsteins,3401,2022,2,3401,wallup,-36.360516,142.245009
286038,zumsteins,3401,2022,2,3401,wartook,-37.054612,142.343663
286039,zumsteins,3401,2022,2,3401,wombelano,-37.015700,141.594000
286040,zumsteins,3401,2022,2,3401,wonwondah,-36.917261,142.257961


In [15]:
# Check if there is Suburb/Town Name that didn't get a macth
crime_count_n_locality[crime_count_n_locality.isnull().any(axis=1)]

,Suburb/Town Name,Postcode,Year,Incidents Recorded,postcode,locality,Lat_precise,Long_precise


#### Then compute similarity score between locality and Suburb/Town Name, and select the matched data/row with highest simialrity score for each (Suburb/Town Name)

In [16]:
# Create a new column name similarityscore that store similarlity score between locality and Suburb/Town Name
crime_count_n_locality['SimilarityScore'] = crime_count_n_locality.apply(
    lambda row: fuzz.ratio(row['Suburb/Town Name'], row['locality']),
    axis=1
)

In [17]:
crime_count_n_locality

,Suburb/Town Name,Postcode,Year,Incidents Recorded,postcode,locality,Lat_precise,Long_precise,SimilarityScore
0,abbeyard,3737,2015,4,3737,abbeyard,-37.021325,146.764065,100
1,abbeyard,3737,2015,4,3737,barwidgee,-36.509837,146.777018,47
2,abbeyard,3737,2015,4,3737,buffalo river,-36.618382,146.681521,29
3,abbeyard,3737,2015,4,3737,dandongadale,-36.911853,146.699920,20
4,abbeyard,3737,2015,4,3737,gapsted,-36.514069,146.656295,40
...,...,...,...,...,...,...,...,...,...
286037,zumsteins,3401,2022,2,3401,wallup,-36.360516,142.245009,13
286038,zumsteins,3401,2022,2,3401,wartook,-37.054612,142.343663,12
286039,zumsteins,3401,2022,2,3401,wombelano,-37.015700,141.594000,33
286040,zumsteins,3401,2022,2,3401,wonwondah,-36.917261,142.257961,11


#### Select the matched locality with highest Simialrity Score with Suburb/Town Name	

In [18]:
# Group by 'Suburb/Town Name' and 'Year' and select row of the data with the highest 'SimilarityScore'
crime_count_n_locality = crime_count_n_locality.loc[crime_count_n_locality.groupby(['Suburb/Town Name','Year'])['SimilarityScore'].idxmax()]


In [19]:
crime_count_n_locality

,Suburb/Town Name,Postcode,Year,Incidents Recorded,postcode,locality,Lat_precise,Long_precise,SimilarityScore
0,abbeyard,3737,2015,4,3737,abbeyard,-37.021325,146.764065,100
15,abbeyard,3737,2017,1,3737,abbeyard,-37.021325,146.764065,100
30,abbeyard,3737,2019,3,3737,abbeyard,-37.021325,146.764065,100
45,abbeyard,3737,2021,1,3737,abbeyard,-37.021325,146.764065,100
60,abbotsford,3067,2014,793,3067,abbotsford,-37.802360,144.998362,100
...,...,...,...,...,...,...,...,...,...
285889,zeerust,3634,2022,2,3634,zeerust,-36.270201,145.397333,100
285897,zeerust,3634,2023,8,3634,zeerust,-36.270201,145.397333,100
285945,zumsteins,3401,2018,2,3401,zumsteins,-37.118787,142.384236,100
285993,zumsteins,3401,2019,2,3401,zumsteins,-37.118787,142.384236,100


#### Check data with poor Simialrity score

In [20]:
# Check data with mis_matched Suburb/Town Name to locality
mis_matched = crime_count_n_locality[crime_count_n_locality['SimilarityScore'] < 100]
mis_matched

,Suburb/Town Name,Postcode,Year,Incidents Recorded,postcode,locality,Lat_precise,Long_precise,SimilarityScore
59863,cokum,3530,2015,1,3530,culgoa,-35.717845,143.104025,36
59867,cokum,3530,2018,1,3530,culgoa,-35.717845,143.104025,36
59871,cokum,3530,2019,1,3530,culgoa,-35.717845,143.104025,36
59875,cokum,3530,2020,1,3530,culgoa,-35.717845,143.104025,36
59879,cokum,3530,2021,1,3530,culgoa,-35.717845,143.104025,36
83490,eddington,3463,2022,1,3463,maldon,-36.998046,144.068599,40
140476,lake eppalock,3523,2016,1,3523,ladys pass,-36.826039,144.686299,43
140486,lake eppalock,3523,2017,2,3523,ladys pass,-36.826039,144.686299,43
140496,lake eppalock,3523,2018,1,3523,ladys pass,-36.826039,144.686299,43
140506,lake eppalock,3523,2019,1,3523,ladys pass,-36.826039,144.686299,43


By obseriving the data frame we see that there are 18 row of data has mismatched Suburb/Town Name and locality. Which suggest that most data were correcly matched between (Yeahhhhhhhhhhhh).

We can choose to retain these data as they all coming from same postcode region, and their lat and long will be somewhere close to their actual location.

However, as these mis matched data mostly coming from "cokum", ""watchem west", "timbarra", "eddington", "lake eppalock" and "mirimbah" with each only contain 1-2 incident record. Hence we can choose to drop these data out as they will be negligible 

In [21]:
# Retain rows with correctly matched Suburb/Town Name === 'SimilarityScore'=100
crime_count_n_locality = crime_count_n_locality[crime_count_n_locality['SimilarityScore'] == 100]
crime_count_n_locality.reset_index(drop=True, inplace=True)


In [22]:
crime_count_n_locality

,Suburb/Town Name,Postcode,Year,Incidents Recorded,postcode,locality,Lat_precise,Long_precise,SimilarityScore
0,abbeyard,3737,2015,4,3737,abbeyard,-37.021325,146.764065,100
1,abbeyard,3737,2017,1,3737,abbeyard,-37.021325,146.764065,100
2,abbeyard,3737,2019,3,3737,abbeyard,-37.021325,146.764065,100
3,abbeyard,3737,2021,1,3737,abbeyard,-37.021325,146.764065,100
4,abbotsford,3067,2014,793,3067,abbotsford,-37.802360,144.998362,100
...,...,...,...,...,...,...,...,...,...
23023,zeerust,3634,2022,2,3634,zeerust,-36.270201,145.397333,100
23024,zeerust,3634,2023,8,3634,zeerust,-36.270201,145.397333,100
23025,zumsteins,3401,2018,2,3401,zumsteins,-37.118787,142.384236,100
23026,zumsteins,3401,2019,2,3401,zumsteins,-37.118787,142.384236,100


### Feature Selection/Drop duplicate/unnecessary columns 

In [23]:
# Drop the specified columns
crime_count_df = crime_count_n_locality.drop(columns=['postcode', 'locality', 'SimilarityScore'])


In [24]:
crime_count_df

,Suburb/Town Name,Postcode,Year,Incidents Recorded,Lat_precise,Long_precise
0,abbeyard,3737,2015,4,-37.021325,146.764065
1,abbeyard,3737,2017,1,-37.021325,146.764065
2,abbeyard,3737,2019,3,-37.021325,146.764065
3,abbeyard,3737,2021,1,-37.021325,146.764065
4,abbotsford,3067,2014,793,-37.802360,144.998362
...,...,...,...,...,...,...
23023,zeerust,3634,2022,2,-36.270201,145.397333
23024,zeerust,3634,2023,8,-36.270201,145.397333
23025,zumsteins,3401,2018,2,-37.118787,142.384236
23026,zumsteins,3401,2019,2,-37.118787,142.384236


#### Convert the crime_count_n_locality to a geodataframe 

In [25]:
# Convert the dataframe to a geodataframe
geometry = [Point(xy) for xy in zip(crime_count_df['Long_precise'], crime_count_df['Lat_precise'])]
crime_count_gpd = gpd.GeoDataFrame(crime_count_df, geometry=geometry)
# drop lat and long columns
crime_count_gpd = crime_count_gpd.drop(columns=['Long_precise', 'Lat_precise'])


In [26]:
crime_count_gpd

,Suburb/Town Name,Postcode,Year,Incidents Recorded,geometry
0,abbeyard,3737,2015,4,POINT (146.76407 -37.02132)
1,abbeyard,3737,2017,1,POINT (146.76407 -37.02132)
2,abbeyard,3737,2019,3,POINT (146.76407 -37.02132)
3,abbeyard,3737,2021,1,POINT (146.76407 -37.02132)
4,abbotsford,3067,2014,793,POINT (144.99836 -37.80236)
...,...,...,...,...,...
23023,zeerust,3634,2022,2,POINT (145.39733 -36.27020)
23024,zeerust,3634,2023,8,POINT (145.39733 -36.27020)
23025,zumsteins,3401,2018,2,POINT (142.38424 -37.11879)
23026,zumsteins,3401,2019,2,POINT (142.38424 -37.11879)


### Save as a copy in raw

In [27]:
crime_count_gpd.to_file('../data/raw/crime_count_with_point_geo.geojson', driver="GeoJSON")  # Specify the driver based on the desired output format
